In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tf
import time
%matplotlib qt

In [3]:
test = lambda x,y :x+y
test(2,5)

7

In [6]:
img = tf.imread("Site4um.tiff")
plt.plot(np.mean(img, axis = (1,2)))
% time (np.mean(np.mean(img, 1), 1))

In [9]:
%time np.mean(img, axis = (1,2))

Wall time: 998 µs


array([0.00477391, 0.00542724, 0.00688025, 0.0202926 , 0.01054166,
       0.00771563, 0.00783147, 0.01526484, 0.01190137, 0.01120089,
       0.01402828, 0.02365644, 0.02966065, 0.02974189, 0.03174809,
       0.03499492, 0.04145871, 0.052057  , 0.07654981, 0.1205722 ,
       0.19034252, 0.26984605, 0.36429673, 0.44518483, 0.5134946 ,
       0.56122667, 0.59451056, 0.59657323, 0.55306095, 0.491212  ,
       0.46307695, 0.4508953 , 0.42275664, 0.39334798, 0.38607094,
       0.37800118, 0.37706068, 0.37495038, 0.37330264, 0.37025386,
       0.36392963, 0.3555663 , 0.34427273, 0.33261928, 0.3219451 ,
       0.31112072, 0.30605176, 0.31051704, 0.31364533, 0.32008824,
       0.33563355, 0.3463106 , 0.35710588, 0.37389025, 0.3838279 ,
       0.38853076, 0.3934457 ], dtype=float32)

In [2]:
def removeAvgSpecFromStack(im_stack):
    mean_spec = np.mean(np.mean(im_stack, 1), 1)
    mean_array = np.ones((im_stack.shape))
    for n, val in enumerate(mean_spec):
        mean_array[n] *= val

    new_img = np.subtract(im_stack,mean_array)

    return new_img


In [17]:
%time ff = removeAvgSpecFromStack(img)
ff.mean()
plt.imshow(ff[-1])

Wall time: 6.98 ms


In [14]:
def removeAvgSpecFromStack2(im_stack):
    mean_spec = np.mean(np.mean(im_stack, 1), 1)
    en, im1, im2 = np.shape(im_stack)
    im_array = im_stack.reshape(en, im1 * im2)
    new_img = np.subtract(im_array,mean_spec[:,np.newaxis])

    return np.reshape(new_img, (en, im1, im2))

In [15]:
%time ff = removeAvgSpecFromStack2(img)
ff.mean()

Wall time: 2.97 ms


-2.8488398e-09

In [16]:
plt.imshow(ff[-1])

In [23]:
from scipy.signal import savgol_filter
def smoothen(image_array, w_size=5):
    a, b, c = np.shape(image_array)
    spec2D_Matrix = np.reshape(image_array, (a, (b * c)))
    smooth_stack = np.zeros(np.shape(spec2D_Matrix))
    tot_spec = np.shape(spec2D_Matrix)[1]

    for i in range(tot_spec):
        norm_spec = spec2D_Matrix[:, i]
        if norm_spec.sum() > 0:
            norm_spec = savgol_filter(norm_spec, w_size, w_size - 2)
        smooth_stack[:, i] = norm_spec

    norm_stack = np.reshape(smooth_stack, (a, b, c))
    return norm_stack

In [24]:
%time foo = smoothen(img, w_size=5)

Wall time: 4.3 s


In [32]:
plt.plot(np.mean(np.mean(foo, 1), 1))

In [47]:
a, b, c = np.shape(img)
spec2D_Matrix = np.reshape(img, (a, (b * c)))
foo2 = savgol_filter(spec2D_Matrix, 5, 3, axis = 0 )
np.shape(foo2)

(57, 8645)

In [49]:
plt.plot(np.mean(np.mean(foo, 1), 1))
plt.plot(np.mean(foo2, -1))

In [51]:
plt.plot(np.mean(foo[:,20], 1))
plt.plot(np.mean(foo2[20], -1))